In [ ]:
##########USalign查看蛋白比对结果流程

In [ ]:
%%time
############两两比对###########################################

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有pdb格式文件
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdb' in file :
                L.append(os.path.join(root, file))
    return L


ids="3506/"#进行比对的子文件夹
ph2="path_to_the_files/"+ids
dictseq={}
for i in file_name2(ph2):#注意会遍历文件夹下所有文件，小心存储
    #print(i)
    if "/us/" not in i and "/q/" not in i:#排除其他子文件夹干扰
        name0=i.split("/")[-1]
        dictseq[name0]=1
dictseqf={}
for key1 in dictseq:
    for key2 in dictseq:
        if key1+key2 not in dictseqf and key2+key1 not in dictseqf and key1!=key2:
            dictseqf[key1+key2]=1
            dictseqf[key2+key1]=1
            mingling="USalign -d 1 "+key1+" "+key2+" -o ./us/" +key1.split(".")[0]+"_"+key2.split(".")[0]+" >"+key1.split(".")[0]+"_"+key2.split(".")[0]+"1A.log"
            #设置标记阈值为1A，结果存储文件夹为./us/
            print(mingling)
            os.system(mingling)
        
    
    
        

In [ ]:
%%time

#########################提取比对结果中满足阈值的氨基酸##############################

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有1A.log尾缀的文件
    
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '1A.log' in file :
                L.append(os.path.join(root, file))
    return L


def find_positions(string):
    positions = []
    j=1
    for i, char in enumerate(string):
        if char != '-':
            positions.append((i,j,char))
            j+=1
            
    return positions


ph=ph2+"us/"
dictseq={}
for i in file_name2(ph):#注意会遍历文件夹下所有文件，小心存储
    if "HGPCPOPL_035"  in i and "266gsd2" not in i:#筛选包含目的蛋白的比对结果
        print(i)
        shuang=i.split("/")[-1].split("1A.log")[0]
        dictseq[shuang]={}
        with open(i, 'r') as file:
            files=[]
            for lin in file:files.append(lin)
            
            if "denotes residue pairs" in files[19]:
                if "Name of Structure" in files[8] and "Name of Structure" in files[9]:
                    name1=files[8].split(":")[1].split("/")[-1].split(".")[0].strip()
                    name2=files[9].split(":")[1].split("/")[-1].split(".")[0].strip()
                    listname1=files[20]
                    listname=files[21]
                    listname2=files[22]
                    ln1=find_positions(listname1)
                    ln2=find_positions(listname2)
                    #print(ln2)
                    if len(listname1)==len(listname)==len(listname2):
                        lenn=len(listname1)
                        j=0
                        dictseq[shuang][name1]=[]
                        dictseq[shuang][name2]=[]
                        
                        while j<lenn:
                            if listname[j]!=":":
                                flag=0###################因为都是第一个对齐第二个，其中‘-’出现位置不定，需设置
                                for kk in ln1:
                                    if kk[0]==j:
                                        k1=kk[1]
                                        p1=kk[2]
                                        flag=1
                                if flag==0:
                                    k1="NA"
                                    p1="NA"
                                        
                                flag=0   
                                for kk3 in ln2:
                                    
                                    if kk3[0]==j:
                                        k3=kk3[1]
                                        p3=kk3[2]
                                        flag=1
                                if flag==0:
                                    k3="NA"
                                    p3="NA"
                                
                                dictseq[shuang][name1].append((listname1[j],j,k1,p1))
                                #print(k1,k2)
                                dictseq[shuang][name2].append((listname2[j],j,k3,p3))
                            j+=1

                    else:print("lenn eorr!")
                else:print("name eorr!")
            else:print("geshi eorr!")
            
            

        
    
        


In [ ]:
%%time
###################################
from Bio.PDB import PDBParser
from Bio.PDB.PDBIO import PDBIO
import pandas as pd
import numpy as np
import os 
import cv2
import re

def convert_to_3letter(aa):
    aa_dict = {
        'A': 'Ala',
        'R': 'Arg',
        'N': 'Asn',
        'D': 'Asp',
        'C': 'Cys',
        'E': 'Glu',
        'Q': 'Gln',
        'G': 'Gly',
        'H': 'His',
        'I': 'Ile',
        'L': 'Leu',
        'K': 'Lys',
        'M': 'Met',
        'F': 'Phe',
        'P': 'Pro',
        'S': 'Ser',
        'T': 'Thr',
        'W': 'Trp',
        'Y': 'Tyr',
        'V': 'Val'
    }

    if aa in aa_dict:
        result=aa_dict[aa].upper()
    else:
        result='Unk' # 未知氨基酸的缩写为Unk
    
    return result
def file_name2(file_dir): #识别路径目录下包含的所有'.pdbqt' 格式文件

    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if '.pdbqt' in file:
                L.append(os.path.join(root, file))
    return L



pdb_files = 'path_to_the_files/'  # 蛋白结构的PDB文件


for pdb_file in file_name2(pdb_files):
    pdb=pdb_file.split("/")[-1].split(".")[0]
    #print(pdb)
    with open(pdb_file, 'r') as file:
            files=[]
            for lin in file:files.append(lin)
    for pdbus in dictseq:
        #print(pdbus)
        if pdb !="":#pdb in pdbus
            for us in dictseq[pdbus]:
                if pdb in us:
                    print(us)
                    sites=dictseq[pdbus][us]
                    f=open("path_to_the_files/shaixuan/"+pdbus+"@_@"+pdb+"1A.pdbqt","w")#写入比对齐的文件
                    nn=0
                    temp=""
                    for a in files:
                        b = re.split(r"\s+", a)
            
                        if len(b)>5 :
                            if len(b[4])>1:
                                temp0=b[3]+b[4]
                            else:temp0=b[3]+b[4]+b[5]
                            if temp0!=temp:
                                nn+=1
                                temp=temp0
                            for kkk in sites:
                                if kkk[2]==nn and kkk[-1]!="\n" and kkk[-1]!="NA":###########algin的fasta文件比pdb文件少一个氨基酸，注意##########
                                    if convert_to_3letter(kkk[0])!=b[3]:
                                        print(kkk,nn,convert_to_3letter(kkk[0]),b[3])
                                        print(a)
                                    f.write(a)
                
                
                
                        if "TER" in a[:5]:
                            f.write(a)
                    f.close()
                        
        
        


In [ ]:
f=open("35061A.csv","w")
for key in dictseq:
    keys_list = list(dictseq[key].keys())
    first_key = keys_list[0]
    second_key = keys_list[1]   
    f.write(key+","+first_key+","+second_key)
    if len(dictseq[key][first_key])!=len(dictseq[key][second_key]):print("WHO ARE YOU!")
    else:
        i=0
        while i<len(dictseq[key][first_key]):
            if dictseq[key][first_key][i][-1]!="\n" or dictseq[key][second_key][i][-1]!="\n":
                f.write(","+str(dictseq[key][first_key][i][-2])+":"+dictseq[key][first_key][i][-1]+":@:"+str(dictseq[key][second_key][i][-2])+":"+dictseq[key][second_key][i][-1])
            i+=1
    f.write("\n")
f.close()